# Question 1: Scrape a Wikipedia page

In [1]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Read a list of dataframe
tables = pd.read_html(url)
df0 = tables[0]
#df0 = df0.set_index("Postal Code")
df0.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [2]:
# Ignore cells with a borough that is Not assigned
df = df0[df0['Borough'] != 'Not assigned']

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighbourhood'] == 'Not assigned' , 'Neighbourhood'] = df['Borough']


df.head(10)



/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [3]:
# More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma
temp_df = df.groupby(['Postal Code','Borough'], sort=False).agg( ', '.join)
temp_df.reset_index(inplace = True)

temp_df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
# Use the .shape method to print the number of rows of your dataframe.
temp_df.shape

(103, 3)

# Question 2: Geospatial Data

In [5]:
# Download the CSV file
!wget -q -O "Toronto.csv"  http://cocl.us/Geospatial_data

# Read the csv file
df_toronto = pd.read_csv("Toronto.csv")
df_toronto.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [6]:
# Join the two tables by using Postal Code
temp_df.set_index('Postal Code').join(df_toronto.set_index('Postal Code'))

temp_df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
df_merge = pd.merge(temp_df, df_toronto, on = "Postal Code")

df_merge.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [8]:
# Work with only boroughs that contain the word Toronto
only_toronto = df_merge[df_merge["Borough"].str.contains("Toronto")]

only_toronto.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


# Question 3: Neighborhoods in Toronto

In [9]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'KOMVDCBWZS0Y3AFXRQIXOEZHXUM5J4LGZSJF1VYMMWITCXTK' 
CLIENT_SECRET = 'G2NHNKYLVV3YKBT2UVQ34NYTLEQCO2G4WVI3ABXLYVZSOH35' # 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KOMVDCBWZS0Y3AFXRQIXOEZHXUM5J4LGZSJF1VYMMWITCXTK
CLIENT_SECRET:G2NHNKYLVV3YKBT2UVQ34NYTLEQCO2G4WVI3ABXLYVZSOH35


In [10]:
# Assume that we are in St. James Town
latitude = 43.651494
longitude = -79.375418
print(latitude, longitude)

43.651494 -79.375418


#### Let's search for Vietnamese foods

In [11]:
import requests # library to handle requests
from pandas.io.json import json_normalize

search_query = 'Vietnamese'
radius = 2000
print("I want some " + search_query + ' foods!')

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

# Send the GET Request and examine the results
results = requests.get(url).json()

# Get relevant part of JSON and transform it into a pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

I want some Vietnamese foods!


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,4b0177f5f964a520b34222e3,254 Spadina Ave.,CA,Toronto,Canada,at Cameron St.,1803,"[254 Spadina Ave. (at Cameron St.), Toronto ON...","[{'label': 'display', 'lat': 43.65176783974445...",43.651768,-79.397811,M5T 2C2,ON,Xe Lua Vietnamese Cuisine 火車頭,v-1602733396
1,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,4af1c3a4f964a520f2e221e3,252 Spadina Ave.,CA,Toronto,Canada,at Cameron St.,1791,"[252 Spadina Ave. (at Cameron St.), Toronto ON...","[{'label': 'display', 'lat': 43.65167646348121...",43.651676,-79.397656,M5T 2C2,ON,Bun Saigon Vietnamese Restaurant,v-1602733396
2,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,4ed9d9f246907c1b41ff496f,NaN,CA,NaN,Canada,NaN,1801,[Canada],"[{'label': 'display', 'lat': 43.65177209719110...",43.651772,-79.397776,NaN,NaN,Vietnamese cuisine,v-1602733396
3,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,59eccc99dff8150ccdd70355,546 Dundas St W,CA,Toronto,Canada,NaN,1934,"[546 Dundas St W, Toronto ON M5T 1H3, Canada]","[{'label': 'display', 'lat': 43.652748, 'lng':...",43.652748,-79.399377,M5T 1H3,ON,Kim Vietnamese Restaurant,v-1602733396
4,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,55917220498e8523827b3bf6,567 King St. W,CA,Toronto,Canada,Portland St.,2085,"[567 King St. W (Portland St.), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.64459034944851...",43.644590,-79.399486,NaN,ON,Bac Ky Vietnamese Canteen,v-1602733396
5,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,55a1bd74498ef07267b993b2,NaN,CA,NaN,Canada,NaN,2091,[Canada],"[{'label': 'display', 'lat': 43.64457516676050...",43.644575,-79.399557,NaN,NaN,Bac Ky Vietnamese Canteen,v-1602733396
6,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,4ad4c060f964a52056f720e3,350 Spadina Ave.,CA,Toronto,Canada,at St Andrews St.,1918,"[350 Spadina Ave. (at St Andrews St.), Toronto...","[{'label': 'display', 'lat': 43.654434, 'lng':...",43.654434,-79.398883,M5T 2G4,ON,Pho Hung,v-1602733396


Define information of interest and filter dataframe

In [12]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Xe Lua Vietnamese Cuisine 火車頭,Vietnamese Restaurant,254 Spadina Ave.,CA,Toronto,Canada,at Cameron St.,1803,"[254 Spadina Ave. (at Cameron St.), Toronto ON...","[{'label': 'display', 'lat': 43.65176783974445...",43.651768,-79.397811,M5T 2C2,ON,4b0177f5f964a520b34222e3
1,Bun Saigon Vietnamese Restaurant,Vietnamese Restaurant,252 Spadina Ave.,CA,Toronto,Canada,at Cameron St.,1791,"[252 Spadina Ave. (at Cameron St.), Toronto ON...","[{'label': 'display', 'lat': 43.65167646348121...",43.651676,-79.397656,M5T 2C2,ON,4af1c3a4f964a520f2e221e3
2,Vietnamese cuisine,Asian Restaurant,NaN,CA,NaN,Canada,NaN,1801,[Canada],"[{'label': 'display', 'lat': 43.65177209719110...",43.651772,-79.397776,NaN,NaN,4ed9d9f246907c1b41ff496f
3,Kim Vietnamese Restaurant,Vietnamese Restaurant,546 Dundas St W,CA,Toronto,Canada,NaN,1934,"[546 Dundas St W, Toronto ON M5T 1H3, Canada]","[{'label': 'display', 'lat': 43.652748, 'lng':...",43.652748,-79.399377,M5T 1H3,ON,59eccc99dff8150ccdd70355
4,Bac Ky Vietnamese Canteen,Vietnamese Restaurant,567 King St. W,CA,Toronto,Canada,Portland St.,2085,"[567 King St. W (Portland St.), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.64459034944851...",43.644590,-79.399486,NaN,ON,55917220498e8523827b3bf6
5,Bac Ky Vietnamese Canteen,Vietnamese Restaurant,NaN,CA,NaN,Canada,NaN,2091,[Canada],"[{'label': 'display', 'lat': 43.64457516676050...",43.644575,-79.399557,NaN,NaN,55a1bd74498ef07267b993b2
6,Pho Hung,Vietnamese Restaurant,350 Spadina Ave.,CA,Toronto,Canada,at St Andrews St.,1918,"[350 Spadina Ave. (at St Andrews St.), Toronto...","[{'label': 'display', 'lat': 43.654434, 'lng':...",43.654434,-79.398883,M5T 2G4,ON,4ad4c060f964a52056f720e3


In [13]:
dataframe_filtered.name

0       Xe Lua Vietnamese Cuisine 火車頭
1    Bun Saigon Vietnamese Restaurant
2                  Vietnamese cuisine
3           Kim Vietnamese Restaurant
4           Bac Ky Vietnamese Canteen
5           Bac Ky Vietnamese Canteen
6                            Pho Hung
Name: name, dtype: object

In [14]:
# install Folium
! pip install folium==0.5.0

In [15]:
import folium

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='I am in St. James Town',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Explore a Given Venue

In [16]:
# Let's explore the closest Vietnamese restaurant
venue_id = '4af1c3a4f964a520f2e221e3' # ID of Bun Saigon Vietnamese Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# Send GET request for result
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


In [17]:
# Get the number of tips

result['response']['venue']['tips']['count']

16

In [18]:
# Get the venue's tips
## Ecco Tips
limit = 20 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()

# Get tips and list of associated features
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]

# Format column width and display all tips
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"Hainan chicken is my new favorite, rice is so good and comes with generous portion of greens",0,0,56205a62498e07083e40f263,Laura,S,65237515


## Search a Foursquare User

In [19]:

idnumber = '65237515' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
#user_data = results['response']['user']
user_data=results['response']['tips']['items'][0]['venue']['photos']['groups'][0]['items']#['items']
# display features associated with user
#user_data.keys()
#results
pd.set_option('display.max_colwidth', -1)

users_df = json_normalize(user_data)

# filter columns
filtered_columns = ['id','user.id','user.firstName','user.lastName']
tips_filtered = users_df.loc[:, filtered_columns]

# display user's tips
df=tips_filtered
df

,id,user.id,user.firstName,user.lastName
0,522a91c711d2b2eecea62955,51899299,Jon,T
1,5163a2d6e4b0d5dc967a20ae,2409845,Kaew,J
2,51d8daa7498ef1b3a6dc36b0,10494109,bbb,i


In [20]:

g=df.loc[df['user.id'] == '51899299']
print('First Name: ' + g['user.firstName'])
print('Last Name: ' + g['user.lastName'])


0    First Name: Jon
Name: user.firstName, dtype: object
0    Last Name: T
Name: user.lastName, dtype: object


Laura is very active on Foursquare too!

In [21]:
# define tips URL
user_id='65237515'
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,Line ups are insane. But sushi is great price and quality,1,0,5b0b7b3c04d1ae002cf6f1c5
1,We just come here for the cremia,0,0,5b0b7b1f180b91002c509abf
2,Very convenient location with the tram stop just outside. Modern rooms and a lovely rooftop pool.,2,0,5a831d1b364d97670964697b
3,Cute spot. Order the guac!,0,0,5a831cedc530933f90bb6157
4,The rooms are super spacious. But the heaters are so noisy! It sounds like someone is vacuuming the floor above. Impossible to sleep :(,0,0,5a831cca78782c427a1800df
5,Crispy pork is a winner!,0,0,5a73db99febf31705c96b2cd
6,Super bowl congee is always good,0,0,5a73db4583e3800645423d46
7,Very friendly staff! They want you to have the best experience possible,1,0,5a73db282f97ec2e8e0c3617
8,Food for Winterlicious was good. Service was a bit unfriendly.,0,0,5a73dabe82a7502a3d7044e3
9,Pine suite was incredible. Amazing service and food! Borrow bikes and ride around the trails in the summer,0,0,59cd4710e679bc7d4e77133c


Explore location around me

In [22]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

results = requests.get(url).json()

items = results['response']['groups'][0]['items']

dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Gyu-Kaku Japanese BBQ,Japanese Restaurant,81 Church St,CA,Toronto,Canada,at Adelaide St E,30,"[81 Church St (at Adelaide St E), Toronto ON M5C 2G2, Canada]","[{'label': 'display', 'lat': 43.651422275497914, 'lng': -79.37504693687086}]",43.651422,-79.375047,NaN,M5C 2G2,ON,574ad72238fa943556d93b8e
1,Fahrenheit Coffee,Coffee Shop,120 Lombard St,CA,Toronto,Canada,at Jarvis St,238,"[120 Lombard St (at Jarvis St), Toronto ON M5C 3H5, Canada]","[{'label': 'display', 'lat': 43.65238358726612, 'lng': -79.37271903848271}]",43.652384,-79.372719,NaN,M5C 3H5,ON,4fff1f96e4b042ae8acddca5
2,GoodLife Fitness Toronto 137 Yonge Street,Gym,137 Yonge St,CA,Toronto,Canada,at Richmond St,215,"[137 Yonge St (at Richmond St), Toronto ON M5C 1W6, Canada]","[{'label': 'display', 'lat': 43.651242, 'lng': -79.378068}]",43.651242,-79.378068,NaN,M5C 1W6,ON,4dc842f2e4cd5bc076711d90
3,Hogtown Smoke,Food Truck,1959 Queen St E,CA,Toronto,Canada,NaN,252,"[1959 Queen St E, Toronto ON M4L 1H7, Canada]","[{'label': 'display', 'lat': 43.64928693557788, 'lng': -79.37468932637938}]",43.649287,-79.374689,NaN,M4L 1H7,ON,4fcb97f7e4b00a0e8520b055
4,GEORGE Restaurant,Restaurant,111c Queen St East,CA,Toronto,Canada,Church,220,"[111c Queen St East (Church), Toronto ON M5C 1S2, Canada]","[{'label': 'display', 'lat': 43.65334645635036, 'lng': -79.3744449051126}]",43.653346,-79.374445,NaN,M5C 1S2,ON,4af618daf964a520220122e3
5,Mystic Muffin,Middle Eastern Restaurant,113 Jarvis St,CA,Toronto,Canada,at Richmond St E,248,"[113 Jarvis St (at Richmond St E), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.652483550499326, 'lng': -79.37265472845176}]",43.652484,-79.372655,NaN,NaN,ON,4b72f550f964a5202e922de3
6,Elgin And Winter Garden Theatres,Theater,189 Yonge St,CA,Toronto,Canada,btwn Queen St E & Shuter St,326,"[189 Yonge St (btwn Queen St E & Shuter St), Toronto ON M5B 2H1, Canada]","[{'label': 'display', 'lat': 43.653393796019586, 'lng': -79.3785073962175}]",43.653394,-79.378507,NaN,M5B 2H1,ON,4ad4c062f964a520b5f720e3
7,Richmond Station,American Restaurant,1 Richmond Street West,CA,Toronto,Canada,Yonge Street,310,"[1 Richmond Street West (Yonge Street), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65156872562108, 'lng': -79.37926594000328}]",43.651569,-79.379266,NaN,NaN,ON,506db1a9e4b0a3f3b31412f0
8,Dineen Coffee,Café,140 Yonge St,CA,Toronto,Canada,at Temperance St,291,"[140 Yonge St (at Temperance St), Toronto ON M5C 1X6, Canada]","[{'label': 'display', 'lat': 43.650497, 'lng': -79.378765}]",43.650497,-79.378765,Financial District,M5C 1X6,ON,514627d1e4b0dba1b85e9ba8
9,Crepe TO,Creperie,52 Church Street,CA,Toronto,Canada,NaN,172,"[52 Church Street, Toronto ON M5C 3C8, Canada]","[{'label': 'display', 'lat': 43.650063279511016, 'lng': -79.37458664523871}]",43.650063,-79.374587,NaN,M5C 3C8,ON,583f54320923922e8141d360


## Let's visualize these items on the map around our location

In [23]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='I stay here',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## Explore Trending Venues

In [24]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f87c580a106903c9429e9ef'},
 'response': {'venues': []}}

In [25]:
# Check if any venues are trending at this time
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [26]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'